In [ ]:
import torch
import numpy as np
from rfm import LaplaceRFM


torch.set_default_dtype(torch.float32)
torch.manual_seed(0)

# define target function
def fstar(X):
    return torch.cat([
        (X[:, 0]  > 0)[:,None],
        (X[:, 1]  < 0.1)[:,None]],
        axis=1).type(X.type())

# create low rank data
n = 4000
d = 100
torch.manual_seed(0)
X_train = torch.randn(n,d)
X_test = torch.randn(n,d)
print(X_train.shape)

y_train = fstar(X_train)
y_test = fstar(X_test)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

X_train_numpy = X_train.numpy()




model = LaplaceRFM(bandwidth=1., diag=False, centering=False)
model.fit(
    (X_train, y_train), 
    (X_test, y_test), 
    method='lstsq', epochs=15,
    iters=5,
    classification=True,
    verbose=True
) 


c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([4000, 100])
Time taken to prefit nmf with 70000 points: 0.1890251636505127 seconds
Using NMF-based projection with rank=2, eta=0.0003, bs=4000


100%|██████████| 1/1 [00:00<00:00,  6.19it/s]

Using NMF projection
grad shape: torch.Size([4000, 2]), kmat shape: torch.Size([4000, 4000])
W_nmf shape: torch.Size([4000, 2]), H_nmf shape: torch.Size([2, 4000]), S_nmf shape: torch.Size([2, 2])


(0 epochs, 0.2s)	train mse: 0.0000 	val mse: 0.5129	train acc: 0.6230 	val acc: 0.4397
New best multiclass-acc: 0.4397
Round 0, Test Acc: 43.97%
Round 0, Test MSE: 0.5129
Optimal M batch size:  669
Using batch size of 669
Sampling AGOP on maximum of 20070 total points


...

Using NMF-based projection with rank=2, eta=0.0003, bs=4000


100%|██████████| 1/1 [00:00<00:00, 114.90it/s]

Using NMF projection
grad shape: torch.Size([4000, 2]), kmat shape: torch.Size([4000, 4000])
W_nmf shape: torch.Size([4000, 2]), H_nmf shape: torch.Size([2, 4000]), S_nmf shape: torch.Size([2, 2])


(0 epochs, 0.0s)	train mse: 0.0000 	val mse: 0.0642	train acc: 0.9970 	val acc: 0.7505
New best multiclass-acc: 0.7505
Final MSE: 0.0642
Final Test Acc: 75.05%


tensor(0.0642, device='cuda:0')

In [7]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory // 1024**3 - 1
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8

In [8]:
print(DEVICE, DEV_MEM_GB)

cuda 2
